In [0]:
from pyspark.sql.types import StructField, StructType, StringType, LongType, DoubleType, TimestampType

# Task 0 Load the “/databricks-datasets/definitive-guide/data/retail-data/all/*.csv” file into a dataframe.

mySchema = StructType([
    StructField("InvoiceNo",LongType(),True),
    StructField("StockCode",StringType(),True),
    StructField("Description",StringType(),True),
    StructField("Quantity",StringType(),True),
    StructField("InvoiceDate",TimestampType(),True),
    StructField("UnitPrice",DoubleType(),True),
    StructField("CustomerID",LongType(),True),
    StructField("Country",StringType(),True)])

df = spark.read.format("csv").schema(mySchema).load("/databricks-datasets/definitive-guide/data/retail-data/all/*.csv", header='true', timestampFormat = "M/d/y h:m")
df.show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows



In [0]:
# Task 1 (20 pts). Count the number of items/StockCode for each InvoiceNo.

from pyspark.sql.functions import col, countDistinct

# Using countDistinct so that one type of item is counted only once

df.groupBy(col("InvoiceNo")).agg(countDistinct(col("StockCode")).alias("Number_of_unique_items")).show(5)

+---------+----------------------+
|InvoiceNo|Number_of_unique_items|
+---------+----------------------+
|   540168|                    51|
|   540260|                    41|
|   538809|                    11|
|   541864|                    19|
|   537381|                    24|
+---------+----------------------+
only showing top 5 rows



In [0]:
# Task 2 (20 pts). For each customer, calculate the total amount of spendings.

from pyspark.sql.functions import col, sum

# Generate spending for single row = unit price * qty
# Group by the customer id and take sum of all the spendings

df.withColumn("Spending",col("UnitPrice") * col("Quantity")).groupBy(col("CustomerID")).agg(sum(col("Spending")).alias("Total_spendings")).show(5)

+----------+------------------+
|CustomerID|   Total_spendings|
+----------+------------------+
|     15194| 7521.170000000001|
|     17048|            864.32|
|     13098|28658.879999999997|
|     16781|            294.65|
|     12967|           1194.75|
+----------+------------------+
only showing top 5 rows



In [0]:
# Task 3 (20 pts). For each item/StockCode, list the total amount of sold products.

# Get single amount = unit price * qty
# Group by stockcode and get total of all the amounts

df.withColumn("Amount",col("UnitPrice") * col("Quantity")).groupBy(col("StockCode")).agg(sum(col("Amount")).alias("Total_amount")).show(5)

+---------+------------------+
|StockCode|      Total_amount|
+---------+------------------+
|    22728|20548.399999999998|
|    21889| 8829.790000000003|
|   90210B|             61.56|
|    21259| 7014.750000000002|
|    21894| 908.3299999999999|
+---------+------------------+
only showing top 5 rows



In [0]:
# Task 4 (20 pts). For each InvoiceDate (Attention: not timestamp), calculate the difference of total sales amount with the previous date. 
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
from pyspark.sql.functions import window, lag
from pyspark.sql.window import Window

# Get single Sale value = unit price * qty for each row. 
# Group by invoice date (using "window" to get values for 1 day) and calculate sum of the sales
# This gives columns INVOICE DATE and TOTAL SALE (for that date)

new_df = df.withColumn("Sale", col("UnitPrice") * col("Quantity")).groupBy(window(col("InvoiceDate"), "1 day")).agg(sum(col("Sale"))).withColumn('Invoice_date',col('window')['start'].cast('date')).select("Invoice_date", col("sum(Sale)").alias("Total_sale"))

# Order by INVOICE DATE to get previous value and use lag to get previous value for TOTAL SALE

lagCol = lag(col("Total_sale"), 1).over(Window.orderBy("Invoice_date"))

# Get SALE_DIFFERENCE = current value - previous day value
# Show columns INVOICE DATE, TOTAL SALE (for that date), PREVIOUS DATE SALE and SALE_DIFFERENCE

new_df.withColumn("Prev_day_sale", lagCol).na.fill(0).withColumn("Sale_difference", col("Total_sale") - col("Prev_day_sale")).show(5)


+------------+------------------+-----------------+-------------------+
|Invoice_date|        Total_sale|    Prev_day_sale|    Sale_difference|
+------------+------------------+-----------------+-------------------+
|  2010-12-01| 58635.56000000026|              0.0|  58635.56000000026|
|  2010-12-02| 46207.27999999991|58635.56000000026|-12428.280000000348|
|  2010-12-03|  45620.4599999999|46207.27999999991| -586.8200000000143|
|  2010-12-05| 31383.95000000016| 45620.4599999999|-14236.509999999736|
|  2010-12-06|53860.180000000015|31383.95000000016| 22476.229999999854|
+------------+------------------+-----------------+-------------------+
only showing top 5 rows



In [0]:
# Task 5 (20 pts). For each InvoiceDate (Attention: not timestamp), calculate the difference of the purchase countries with the next date. 

from pyspark.sql.functions import collect_set, lead, array_except

# Group by invoice date (using "window" to get values for 1 day) and get set of countries (collect_set removes duplicates)
# This gives columns INVOICE DATE and COUNTRIES (array)

new_df = df.groupBy(window(col("InvoiceDate"), "1 day")).agg(collect_set(col("Country"))).withColumn('Invoice_date',col('window')['start'].cast('date')).select("Invoice_date", col("collect_set(Country)").alias("Countries"))

# Order by INVOICE DATE to get next value and use lead to get next value for COUNTRIES

leadCol = lead(col("Countries"), 1).over(Window.orderBy("Invoice_date"))

# Get Country difference = {Next day countries} - {Countries} using array_except
# Show columns INVOICE DATE, Countries, Next_day_countries, Country_difference

new_df.withColumn("Next_date_countries", leadCol).withColumn("Country_difference", array_except(col("Countries"),col("Next_date_countries"))).show(5)

+------------+--------------------+--------------------+--------------------+
|Invoice_date|           Countries| Next_date_countries|  Country_difference|
+------------+--------------------+--------------------+--------------------+
|  2010-12-01|[France, Australi...|[EIRE, Germany, U...|[France, Australi...|
|  2010-12-02|[EIRE, Germany, U...|[France, Portugal...|                  []|
|  2010-12-03|[France, Portugal...|[France, Germany,...|[Portugal, Italy,...|
|  2010-12-05|[France, Germany,...|[Portugal, Italy,...|[France, Japan, L...|
|  2010-12-06|[Portugal, Italy,...|[France, Germany,...|[Portugal, Italy,...|
+------------+--------------------+--------------------+--------------------+
only showing top 5 rows

